In [1]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [1,039 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:1

In [2]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-12-01 00:39:54--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.30MB/s    in 0.2s    

2022-12-01 00:39:55 (5.30 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [6]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles
url = 'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz'
spark.sparkContext.addFile(url)
music_df = spark.read.csv(SparkFiles.get('amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz'), sep='\t', header=True)
music_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   45610553| RMDCHWD0Y5OZ9|B00HH62VB6|     618218723|AGPtek® 10 Isolat...|Musical Instruments|          3|            0|          1|   N|                N|         Three Stars|Works very good, ...| 2015-08-31|
|         US|   14640079| RZSL0BALIYUNU|B003LRN53I|     986692292|Sennheiser HD203 ...|Musical Instruments| 

In [7]:
# Get the number of rows in the DataFrame.
music_df.count()

904765

# Transform the Data

## Create the "review_id_table".

In [8]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.
review_id_df = music_df.select(['review_id', 'customer_id', 'product_id', 'product_parent', 'review_date'])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RMDCHWD0Y5OZ9|   45610553|B00HH62VB6|     618218723| 2015-08-31|
| RZSL0BALIYUNU|   14640079|B003LRN53I|     986692292| 2015-08-31|
| RIZR67JKUDBI0|    6111003|B0006VMBHI|     603261968| 2015-08-31|
|R27HL570VNL85F|    1546619|B002B55TRG|     575084461| 2015-08-31|
|R34EBU9QDWJ1GD|   12222213|B00N1YPXW2|     165236328| 2015-08-31|
|R1WCUI4Z1SIQEO|   46018513|B001N4GRGS|     134151483| 2015-08-31|
| RL5LNO26GAVJ1|   10225065|B009PJRMHQ|     694166585| 2015-08-31|
|R3GYQ5W8JHP8SB|    6356995|B00NKBDAZS|     446431775| 2015-08-31|
|R30SHYQXGG5EYC|   35297198|B006MIU7U2|     125871705| 2015-08-31|
|R14YLXA56NP51I|   32139520|B000FIBD0I|     771888534| 2015-08-31|
|R1ZH0HSH38IOTZ|   36060782|B0002E52GG|      68535945| 2015-08-31|
|R3H53KLLC210XI|    5301309|B00RZIH52G|     725541773| 2015-08

## Create the "products" Table

In [10]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
products_df = music_df.select(['product_id', 'product_title']).drop_duplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B003WZXSWM|Grover 205BC Mini...|
|B00ASHQQKI|SUPERNIGHT 5050 1...|
|B000T4BC1I|Aulos A211A/A511B...|
|B005I0HTCY|Neewer Black 3.5m...|
|B00JJ18VIE|Reliable Hardware...|
|B00US77MXI|Pick Geek Guitar ...|
|B00S5O7HZO|Glory B Flat Clar...|
|B007CPU1A0|Meinl Headliner T...|
|B0002E3D9Y|PACK A STAND Clar...|
|B00J3W1F96|Zildjian 5A Acorn...|
|B0006NMUJU|Shure Foam Windsc...|
|B0018SJLFS|Tung-Sol 12AX7 Pr...|
|B00N0HXT94|Akai Professional...|
|B0006GRRQI|Gator G-PG ACOUST...|
|B005DJOI8I|Neewer 3.5mm Hand...|
|B000WS1QC6|Yamaha PA130 120 ...|
|B00IEQ4CVK|4/4 Size Electric...|
|B0080GZPRY|VocoPro CARRY-OKE...|
|B00I4ZMW2C|San Fransisco Gia...|
|B000F9NBZC|Tapco by Mackie M...|
+----------+--------------------+
only showing top 20 rows



## Create the "customers" Table

In [11]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
customers_df = music_df.groupBy('customer_id').count()
customers_df.show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|    9163460|    2|
|   50809283|    1|
|   13005724|    1|
|   51506042|    1|
|   48752158|   22|
|   12318800|    2|
|   26246534|    2|
|   47577568|    1|
|   15963400|   10|
|     134314|    1|
|   32457519|    1|
|   23389022|    9|
|   32604233|    1|
|    3430931|    8|
|   22387890|    2|
|     847650|    1|
|   14633838|    3|
|   11886080|    1|
|    2664669|    1|
|   23389273|    1|
+-----------+-----+
only showing top 20 rows



## Create the "vine_table".

In [12]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
vine_df = music_df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RMDCHWD0Y5OZ9|          3|            0|          1|   N|
| RZSL0BALIYUNU|          5|            0|          0|   N|
| RIZR67JKUDBI0|          3|            0|          1|   N|
|R27HL570VNL85F|          5|            0|          0|   N|
|R34EBU9QDWJ1GD|          5|            0|          0|   N|
|R1WCUI4Z1SIQEO|          5|            0|          0|   N|
| RL5LNO26GAVJ1|          2|            3|          4|   N|
|R3GYQ5W8JHP8SB|          5|            0|          0|   N|
|R30SHYQXGG5EYC|          5|            0|          0|   N|
|R14YLXA56NP51I|          5|            1|          1|   N|
|R1ZH0HSH38IOTZ|          5|            0|          0|   N|
|R3H53KLLC210XI|          4|            0|          0|   N|
|R3OOR877NGA8JK|          3|            0|          0|   N|
|R1BY7WKOZ3KMH0|          2|            

# Load

In [13]:
mode = "append"
jdbc_url="jdbc:postgresql://<endpoint>:5432/my_data_class_db"
config = {"user":"postgres", "password": "<password>", "driver":"org.postgresql.Driver"}

In [14]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='reviews', mode=mode, properties=config)

Py4JJavaError: ignored

In [15]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

Py4JJavaError: ignored

In [ ]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine', mode=mode, properties=config)